In [1]:
import torch
from torch import nn
from torch.nn import NLLLoss
from torch.optim import Adam, SparseAdam
import numpy as np
import pickle
from icecream import ic
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home2/abhisingam/miniconda3/envs/inlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class AMORE (nn.Module):
    def __init__(self):
        super(AMORE, self).__init__()
        self.epochs = 0
        self.word_input_size = 300
        self.speaker_input_size = 65
        self.speaker_embedding_size = 134
        self.lstm_hidden_size = 459        
        
        self.SpeakerEmbed = nn.Embedding(self.speaker_input_size+1, self.speaker_embedding_size, padding_idx = self.speaker_input_size)
        self.Dropout1 = nn.Dropout(8e-3)
        self.Activation = nn.Tanh()
        self.LSTM = nn.LSTM(input_size = self.word_input_size + self.speaker_embedding_size, hidden_size = self.lstm_hidden_size, bidirectional=True)
        self.Dropout2 = nn.Dropout(13e-4)
        self.Classifier = nn.Linear(2*self.lstm_hidden_size, self.speaker_embedding_size)
        self.Cos = nn.CosineSimilarity(dim = 2)
        self.Softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, x):
        x0 = [i[0] for i in x[0]]
        words = torch.stack(x0)
        words = words.to(device)
        
        speak_embed = torch.sum(self.SpeakerEmbed(torch.LongTensor(x[1]).to(device)),dim=1)
        
        embed = torch.cat((words,speak_embed),dim=1)
        embed = self.Dropout1(embed)
        embed = self.Activation(embed)
        
        h1, _= self.LSTM(embed)
        h1 = self.Dropout2(h1)
        
        e1 = self.Classifier(h1)
        
        o1 = self.Cos(e1.unsqueeze(1),self.SpeakerEmbed.weight)
        o2 = self.Softmax(o1)
        
        return o2
        

In [3]:
with open("../output/train_padded.pkl", "rb") as f:
    train_loader = pickle.load(f)
with open("../output/test_padded.pkl", "rb") as f:
    test_loader = pickle.load(f)
with open("../output/val_padded.pkl", "rb") as f:
    val_loader = pickle.load(f)

In [ ]:
# model initialization

model = AMORE()
model.to(device)
loss_func = NLLLoss()
optimizer = Adam(model.parameters(), 5e-4)

In [5]:
# training loop

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    model.epochs += 1
    train_loss = 0
    count = 0
    print("Epoch:", model.epochs)
    for sent in train_loader:
        model.zero_grad()

        X, Y = sent
        
        y = model(X)

        loss = loss_func(y, torch.LongTensor(Y).to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    train_loss /= len(train_loader)
    print("Train Loss:", train_loss)
    
    
    # validation
    
    model.eval()
    val_loss = 0
    targets = []
    preds = []
    for sent in val_loader:
        X, Y = sent
        targets+=[i.tolist() for i in Y]
        y = model(X)
        preds += torch.argmax(y, dim=1).to('cpu')
        loss = loss_func(y, torch.LongTensor(Y).to(device))
        val_loss += loss.item()
    val_loss /= len(val_loader)
    print("Validation Loss:", val_loss)
    # print(targets)
    # print(preds)
    print("F1:", f1_score(targets, preds, average='macro'))
    print()

Epoch: 1
Train Loss: 3.2542030477523802


ic| 'Validation Loss:', val_loss: 3.1594827358539286
ic| "F1:": 'F1:'
    f1_score(targets, preds, average='macro'): 0.03127870264064294


Epoch: 2
Train Loss: 3.0733260560035705


ic| 'Validation Loss:', val_loss: 2.9902295185969425
ic| "F1:": 'F1:'
    f1_score(targets, preds, average='macro'): 0.03127870264064294


Epoch: 3
Train Loss: 2.922109792232513


...

ic| 'Validation Loss:', val_loss: 2.5177575771625227
ic| "F1:": 'F1:'
    f1_score(targets, preds, average='macro'): 0.03127870264064294


Epoch: 27
Train Loss: 2.520907577037811


ic| 'Validation Loss:', val_loss: 2.51750886623676
ic| "F1:": 'F1:'
    f1_score(targets, preds, average='macro'): 0.03127870264064294


Epoch: 28


KeyboardInterrupt: 

In [ ]:
targets[0]

In [ ]:
# testing loop
model.eval()
test_loss = 0
targets = []
preds = []
for sent in test_loader:
    X, Y = sent
    targets += [i.tolist() for i in Y]
    y = model(X)
    
    preds += torch.argmax(y, dim=1).to('cpu')
    
    loss = loss_func(y, torch.LongTensor(Y).to(device))
    test_loss += loss.item()
test_loss /= len(test_loader)
ic("Test Loss:", test_loss)
ic("F1:", f1_score(targets, preds, average='macro'))